In [3]:
import findspark
findspark.init()
import numpy as np
import scipy
from scipy import linalg,dot
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg.distributed import RowMatrix

#this is just a demo array, there will be small changes in this algorithm,
#the whole code will be converted to a function which will have different 
#parameters such as array according to the demand of retailer.

def recommendation(R):
    distR = sc.parallelize(R)
    r_sum = distR.map(lambda x: x.sum()) #row sum 
    c_sum = sc.parallelize(R.transpose()).map(lambda x: x.sum()) #column sum
    
    no_rows = distR.count()
    no_cols = (distR.take(1)[0]).size

    ra = r_sum.map(lambda x: x/no_rows)         #check if this is equal to row_avg
    
    ca = c_sum.map(lambda x: x/no_cols)
    #print(ca.take(3))
    #print(ca.take(3)[1])
    
    #print(distR.map(lambda x: x).map(lambda y:y == 0).count())
    
    R_new1 = np.zeros(shape=(no_rows, no_cols))
    distR_new = sc.parallelize(R_new1)
    for i in range(1, no_rows+1):
        for j in range(0, no_cols):
            if((distR.take(i)[i-1])[j] == 0):
                print(ca.take(j+1)[j])
                #(distR_new.take(i)[i-1])[j] = ca.take(j+1)[j]
            else:
                (distR_new.take(i)[i-1])[j] = (distR.take(i)[i-1])[j]
    #print(distR_new.take(2)[1])
    
    R_nor = np.zeros(shape=(numrows, numcols))       #comment later
    for i in range(0, numrows):                      #comment later
        for j in range(0, numcols):                  #comment later
            R_nor[i][j] = R_new[i][j] - row_avg[i]   #comment later
    #print(R_nor)
    
    R_nor1 = np.zeros(shape=(numrows, numcols))
    distR_nor = sc.parallelize(R_nor1)
    for i in range(1, no_rows+1):
        for j in range(0, no_cols):
            (distR_nor.take(i)[i-1])[j] = (distR_new.take(i)[i-1])[j] - ra.take(i)[i-1]
    #print((distR_nor.take(1)[0]))

    print("-----------------------------------------------------")
    U, S, V = linalg.svd(R_nor, full_matrices=1, compute_uv=1)
    #print(U), print(S), print(V)

    R_red = dot(dot(U, linalg.diagsvd(S, len(S), len(V))) ,V)
    #print(R_red)
    
    U_n, S_n, V_n = linalg.svd(R_red, full_matrices=1, compute_uv=1)
    #print(U_n), print(S_n), print(V_n)

    dimensions = 1
    if dimensions <= numrows:
        for index in range(numrows - dimensions, numrows):
            S[index] = 0
        
    #reducing dimensionality of S 
    S_k = np.zeros((numrows-1, numrows-1))
    for i in range(0, numrows-1):
        S_k[i][i] = S_n[i]
    
    #reducing dimensionality of U 
    U_k = np.zeros((numrows, numrows-1))
    for i in range(0, numrows):
        for j in range(0, numrows-1):
            U_k[i][j] = U_n[i][j]

    #reducing dimensionality of V 
    #print(V_n)
    V_k = np.zeros((numrows-1, numcols))
    for i in range(0, numrows-1):
        for j in range(0, numcols):
            V_k[i][j] = V_n[i][j]
    #print(U_k), print(S_k), print(V_k)
    
    sq_S_k = linalg.sqrtm(S_k)
    #print(sq_S_k)
    
    #print(len(U_k)), print(len(U_k[0])), print(len(S_k)), print(len(S_k[0])), print(len(V_k)), print(len(V_k[0]))
    A = np.matmul(U_k, sq_S_k.transpose())
    B = np.matmul(sq_S_k, V_k)
    #print(B)
    row = len(B)
    col = len(B[0])
    
    #this is not reuired but laterthis is just for testing later should be removed
    R_new = np.zeros((row, col))
    #print(row), print(col)
    for j in range(0, row):
        for f in range(0, col):
            R_new[j][f] = adjusted_cosine_similarity(B, j, f)
    #print(R_new)
    user_a = 1
    item_j = 2
    pr = 0
    u = 0
    v = 0
    #l is not found yet needs to be clarified by alix
    l = [1.02,2.03,3.04,5.06]
    for k in l:
        u = u + adjusted_cosine_similarity(B, item_j, k)* (R_red[user_a][item_j]*row_avg[user_a])
        v = v + np.absolute(adjusted_cosine_similarity(B, item_j, k))
        pr = pr + u/v
        
    #print(pr)
    #result  = prediction(B, user_a=1, item_j=2)
    #return result


def adjusted_cosine_similarity(array, j, f):
    item1 = 0
    item2 = 0
    item3 = 0
    for i in range(0, len(array)):
        item1 = item1 + (array[i][j] * array[i][f])
        item2 = item2 + (array[i][j] * array[i][j])
        item3 = item3 + (array[i][f] * array[i][f])
    result = item1/( np.sqrt(item2*item3) * np.sqrt(item3*item3) )
    return result

def prediction(B, user_a, item_j):
    pr = 0
    u = 0
    v = 0
    #l is not found yet needs to be clarified by alix
    l = [1.02,2.03,3.04,5.06]
    for k in l:
        u = u + adjusted_cosine_similarity(B, item_j, k)* (R_red[user_a][item_j]*row_avg[user_a])
        v = v + np.absolute(adjusted_cosine_similarity(B, item_j, k))
        pr = pr + u/v
    return pr

if __name__ == "__main__":
    
    conf = SparkConf() \
      .setAppName("Recomendation") \
      .set("spark.executor.memory", "2g")
    sc = SparkContext.getOrCreate(conf) 
    R = np.asarray([[1,3,2,5,3,0,2,1,5,3],
               [0,2,1,3,0,4,1,3,5,0],
               [5,0,1,2,4,3,0,4,1,3],
               [4,2,0,3,0,4,5,0,3,2],
               [0,4,1,0,4,5,0,2,0,4]])
    
    recommendation(R)
    sc.stop()

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.


Py4JJavaError: An error occurred while calling o27.load.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:38)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:78)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:78)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:78)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:34)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql.execution.datasources.